<a href="https://colab.research.google.com/github/Rikupro-creator/LLM-prompt-recovery/blob/main/t5_small_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llm-prompt-recovery:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67121%2F7806901%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T151209Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D70aa2e90265a6f2c00bdf47c582a9b204e0ecc56352a2b55b477bc0d711eec990d9b63b4898ba6e602dc264d5c0e6661acfe1a121c0c4ffe45f844cee4857ff35538a9c430e7f1f3c51c3e14f1b5545cbbae8227a98fb9785aedc5c408a38dd1c5cbd49158a05aff9fc806845eb2b196bce2d6e0c5b1f196decc1cca29ee63ecab64bc6239b05a41992eaef3aaf3bfc6e788fd7b7c7c214db982fc4d509420d175db53fbc89545f0143b7a3a5220974d0602b0355fbe16d031fe498ffe95a6c93131df38c5ea5384aa53d0682439a54a82a6d0f9268ba62e5a6b2d04f0a9e7d883d8cb1eefbf12bdeabc3f1433579207b506871ed5124be74f2a069786530563,flan-t5/pytorch/small/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3046%2F4250%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T151210Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D387df6611b5b17e70c4bbbda5852c8db5d6d5e8cd6407fb37efc15a621f51396f7973a0b7e65bbe654f7c22fd6542d8fc1faf7a2908c303e8baebf3bebc5ad33d0693974b8c9182e6a27bf9d794a139e7e9dbad7b854d778634ae161796bac2cbabfb19e953aad320b58fcd588b5e4af7bccd0e7e657e8054b53a9b842ea19c178455266e7792bd2b2da0f534ce19d389aa8c80fd010e9a077a754728d27b25386e35c209a6671f72f0c4740618a26ced34c9a66d4d062bebaf401f52d11faa968725e806df2d43cd1466e85074eb6cd00a2ee2d93356ae13f42005424a4305fb2839c5c49ec0141b6768b2873dafaaa7310185691aeb3c6517c9dfbce27932f,llama-2/pytorch/13b-chat-hf/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3097%2F4302%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T151210Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D52d88b09bbce05f1791abac55f0e6117dcd9c259d8290eb6fcf87fc64b6df1cb541b3c4292c76c30c527e4cbd9b9bea9752aa59dc5156876d192bfca5bc4924e1f83f93899e35318c6ac200ebbbac6403127c8d95a65eb5a094bd38e5ba50c264a09be960f516e0185e1981c9f1304c8f0b9a403ab6fd06cf4993757b241c8612c2f1c7e700969d586803a57aa13f80c810e6389f71f0683fc1e560f378349089b9a052583b485097632ed95f0696b5ee56863b75ef0c2b68157dbddea569227ca7e456bdb4d9bf5770796bc182e62c664f1e2d9cfc58c715cbd1e0d0ce10e0fbf86bda2b4088c735000b24b48306b59de3bb46d963deb72740cefa867db85be'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd


In [ ]:
test = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the directory path where the model and tokenizer are stored
model_directory = "/kaggle/input/flan-t5/pytorch/small/2"

# Load pre-trained T5 model and tokenizer from the local directory
tokenizer = T5Tokenizer.from_pretrained(model_directory)
model = T5ForConditionalGeneration.from_pretrained(model_directory)

question = 'question: provide the form the text has been transformed?'

# Define a function to generate prompts and responses
def generate_prompts(rewritten_text):
    # Define the prompt
    prompt = f"{question}\n{rewritten_text}"

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate response
    outputs = model.generate(**inputs)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Generate responses for each text in the 'rewritten_text' column
responses = ['improve this text into ' + generate_prompts(text) for text in test['rewritten_text']]


In [ ]:
responses

In [ ]:
test['rewritten_text']

In [ ]:
submission=pd.DataFrame({
    'id':test.id,
    'rewrite_prompt':responses
})

In [ ]:
submission.to_csv('submission.csv', index=False)
